In [ ]:
import numpy as np
%tensorflow_version 1.x

import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint
import os
# Keras

from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
# Any results you write to the current directory are saved as output.
# Others
import string

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
from gensim.models import FastText

FastText.

In [ ]:
import sys
!{sys.executable} -m pip install fasttext

     |████████████████████████████████| 71kB 2.2MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3015313 sha256=5e25a5b903dd39bd4e051a4c7481a94a765ca29de8a1a86e48dd52f34f54090e
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
import fasttext.util
fasttext.util.download_model('ta', if_exists='ignore')  # Tamil
ft = fasttext.load_model('cc.ta.300.bin')

Buffered data was truncated after reaching the output size limit.

In [ ]:
import fasttext

ModuleNotFoundError: ignored

In [ ]:


def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix


In [ ]:
import pandas as pd
def load_dataset(filename):
  df = pd.read_csv(filename, encoding = "utf-8",
       names = ["Sentence", "Intent"])
  intent = df["Intent"]
  unique_intent = list(set(intent))
  sentences = list(df["Sentence"])
  
  return (intent, unique_intent, sentences)

In [ ]:
intent, unique_intent, sentences = load_dataset("/content/bank.csv")

In [ ]:
print(intent[:5])

0    2in1_acoount_info
1    2in1_acoount_info
2    2in1_acoount_info
3    2in1_acoount_info
4    2in1_acoount_info
Name: Intent, dtype: object


In [ ]:
nltk.download("punkt")
def cleaning(sentences):
  words = [] 
  for s in sentences:
    w = word_tokenize(s)
    words.append([i for i in w])     
  return words  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
cleaned_words = cleaning(sentences)
print(len(cleaned_words))
print(cleaned_words[:2])  
print(len(sentences))

1567
[['நான்', '2in1', 'கணக்கில்', 'சேமிப்பது', 'எப்படி', '?'], ['2in1', 'கணக்கில்', 'நான்', 'சேமிப்பது', 'எப்படி', '?']]
1567


In [ ]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
  token = Tokenizer(filters = filters)
  token.fit_on_texts(words)
  return token

In [ ]:
def max_length(words):
  return(len(max(words, key = len)))

In [ ]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))

Vocab Size = 1141 and Maximum length = 22


In [ ]:
embedding_dim = 50
embedding_matrix = create_embedding_matrix('/content/cc.ta.300.bin',word_tokenizer.word_index, embedding_dim)

UnicodeDecodeError: ignored

In [ ]:
def encoding_doc(token, words):
  return(token.texts_to_sequences(words))

In [ ]:
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)

In [ ]:
def padding_doc(encoded_doc, max_length):
  return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [ ]:
padded_doc = padding_doc(encoded_doc, max_length)

In [ ]:
padded_doc[:5]

array([[  3,  45,  62, 348,  11,   1,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 45,  62,   3, 348,  11,   1,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 11,   3,  45,  62, 348,   1,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  3,  11,  45,  62, 348,   1,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 45,  62,   3, 587, 287,   2,   1,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0]], dtype=int32)

In [ ]:
print("Shape of padded docs = ",padded_doc.shape)

Shape of padded docs =  (1567, 22)


In [ ]:
#tokenizer with filter changed
output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')

In [ ]:
output_tokenizer.word_index

{'2in1_acoount_info': 11,
 '2in1_atm_card': 15,
 '2in1_interest_receiving': 27,
 '2in1_min_balance': 55,
 '2in1_pass_book': 13,
 'account_currency': 5,
 'bank_statement_online': 41,
 'cancel_card': 54,
 'card_automatic_renewal': 35,
 'card_foreign_use': 28,
 'card_machine_repair': 50,
 'card_usage': 36,
 'change_details': 34,
 'cvv_use': 42,
 'debit_card_requirement': 48,
 'fcaispe_required_docs': 19,
 'foreign_account_lkr_withdrawal': 20,
 'foreign_currency_withdrawal': 3,
 'foreign_currency_withdrawal_currency': 7,
 'foreign_deposit_loan': 29,
 'get_lc_form': 46,
 'get_monthly_report': 38,
 'housing_loan_documents': 1,
 'housing_loan_purpose': 39,
 'interest_credit_info': 52,
 'internet_bank_loan_amount': 30,
 'joint_account_details': 37,
 'life_insurance_limit': 40,
 'loan_requirement': 8,
 'marriage_claim': 43,
 'new_card_reader_cost': 45,
 'new_saving_book': 14,
 'nrfc_account_opening': 9,
 'nrfc_info': 33,
 'precashing_foreign_fixed_deposit': 47,
 'repos_benefits': 53,
 'resident

In [ ]:
encoded_output = encoding_doc(output_tokenizer, intent)

In [ ]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

In [ ]:
encoded_output.shape

(1567, 1)

In [ ]:
def one_hot(encode):
  o = OneHotEncoder(sparse = False)
  return(o.fit_transform(encode))

In [ ]:
output_one_hot = one_hot(encoded_output)

In [ ]:
output_one_hot.shape

(1567, 56)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2)

In [ ]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (1253, 22) and train_Y = (1253, 56)
Shape of val_X = (314, 22) and val_Y = (314, 56)


In [ ]:
try:
    response = requests.get(url)
except Exception as e:
    print(e)




In [ ]:
with open('events.txt','w') as fd:
    fd.write(response.text)

In [ ]:
file = open("embed.tz", "w")


In [ ]:
import gzip
f = gzip.open(r, 'rb')
file_content = f.read()
f.close()

TypeError: ignored

In [ ]:
import gzip
f = gzip.open('file.txt.gz', 'rb')
file_content = f.read()
f.close()

In [ ]:
def create_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 300, input_length = max_length, trainable = False))
  model.add(Bidirectional(LSTM(128)))
#   model.add(LSTM(128))
  model.add(Dense(32, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(56, activation = "softmax"))
  
  return model

In [ ]:
model = create_model(vocab_size, max_length)

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 22, 300)           342300    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_5 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 56)                1848      
Total params: 791,668
Trainable params: 449,368
Non-trainable params: 342,300
_________________________________________________________________


In [ ]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

hist = model.fit(train_X, train_Y, epochs = 100, batch_size = 32, validation_data = (val_X, val_Y), callbacks = [checkpoint])

Train on 1253 samples, validate on 314 samples
Epoch 1/100
1253/1253 [==============================] - 7s 5ms/step - loss: 3.9949 - acc: 0.0471 - val_loss: 3.9353 - val_acc: 0.0701

Epoch 00001: val_loss improved from inf to 3.93529, saving model to model.h5
Epoch 2/100
1253/1253 [==============================] - 3s 3ms/step - loss: 3.9036 - acc: 0.0950 - val_loss: 3.7962 - val_acc: 0.1561

Epoch 00002: val_loss improved from 3.93529 to 3.79616, saving model to model.h5
Epoch 3/100
1253/1253 [==============================] - 3s 3ms/step - loss: 3.6894 - acc: 0.1293 - val_loss: 3.4614 - val_acc: 0.1210

Epoch 00003: val_loss improved from 3.79616 to 3.46139, saving model to model.h5
Epoch 4/100
1253/1253 [==============================] - 4s 3ms/step - loss: 3.3546 - acc: 0.1883 - val_loss: 3.0261 - val_acc: 0.2675

Epoch 00004: val_loss improved from 3.46139 to 3.02610, saving model to model.h5
Epoch 5/100
1253/1253 [==============================] - 4s 3ms/step - loss: 3.0238 - acc

In [ ]:
 model = load_model("model.h5")

In [ ]:
def predictions(text):
  
  test_word = word_tokenize(text)
  test_ls = word_tokenizer.texts_to_sequences(test_word)
  print(test_word)
  #Check for unknown words
  if [] in test_ls:
    test_ls = list(filter(None, test_ls))
    
  test_ls = np.array(test_ls).reshape(1, len(test_ls))
 
  x = padding_doc(test_ls, max_length)
  
  pred = model.predict_proba(x)
  
  
  return pred

In [ ]:
def get_final_output(pred, classes):
  predictions = pred[0]
 
  classes = np.array(classes)
  ids = np.argsort(-predictions)
  classes = classes[ids]
  predictions = -np.sort(-predictions)
 
  for i in range(pred.shape[1]):
    print("%s has confidence = %s" % (classes[i], (predictions[i])))



In [ ]:
text = "எனது புதிய விவரங்களை சேர்க்க முடியுமா?"
pred = predictions(text)
get_final_output(pred, unique_intent)

['எனது', 'புதிய', 'விவரங்களை', 'சேர்க்க', 'முடியுமா', '?']
update_personal_details has confidence = 0.93242615
SLBFE_info has confidence = 0.020040171
change_details has confidence = 0.019487733
cancel_card has confidence = 0.015004314
card_foreign_use has confidence = 0.0035710086
tikiri_required_docs has confidence = 0.003110624
bank_statement_online has confidence = 0.0011076604
selan_sure_info has confidence = 0.00086482265
debit_card_requirement has confidence = 0.0008186766
NRFC_info has confidence = 0.00070686627
life_insurance_limit has confidence = 0.0006872598
get_monthly_report has confidence = 0.00047484663
foreign_account_lkr_withdrawal has confidence = 0.00031837297
interest_credit_info has confidence = 0.00025930084
marriage_claim has confidence = 0.00023986808
2in1_atm_card has confidence = 0.00020179489
card_usage has confidence = 0.00013834114
2in1_interest_receiving has confidence = 8.135061e-05
2in1_acoount_info has confidence = 7.6420656e-05
suspious_activity has c

In [ ]:

print(model.test_on_batch(val_X,val_Y))
model.metrics_names

[0.2949007, 0.9426752]


['loss', 'acc']